## Capitulo 2: NGsolve FEM for elliptic problem

Sea $\Omega$ un dominio en $\mathbb R^{d}$. Considere el operador diferencial $L$
$$
L u = - \nabla\cdot (\sigma \nabla u) + \beta\cdot\nabla u + \mu u
$$
donde $\sigma:\Omega \to \mathbb R^{d,d}$, $\beta:\Omega\to \mathbb R^{d}$, $\mu\in \mathbb R$. Dada una funci\'on $f:\Omega \to \mathbb R$ y considere el problema
de hallar una funci\'on $u:\Omega\to \mathbb R$ tal que
$$
L u =f, \quad \text{en }\Omega,\qquad \mathcal B u =g,\quad \text{sobre }\partial \Omega.
$$

In [16]:
from ngsolve import *
from ngsolve.webgui import Draw

In [105]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.25))
# Draw (mesh);

In [106]:
def H1_ellipticSolver(mesh, order):
    sigma = 0.1
    beta = CF((1.0,1.0))
    mu = 0.5
    
    Vh = H1(mesh, order=order, dirichlet='left|bottom|right|top')
#     HDiv(mesh, order,'RT')
    u = Vh.TrialFunction()
    v = Vh.TestFunction()

    f = LinearForm(Vh)
#     f += 10*v*dx
    f += (sigma*2*pi**2*sin(pi*x)*sin(pi*y) +
          beta[0]*pi*cos(pi*x)*sin(pi*y)+beta[1]*pi*sin(pi*x)*cos(pi*y)+
          mu*sin(pi*x)*sin(pi*y)) *v*dx


    a = BilinearForm(Vh)
    a += sigma*grad(u)*grad(v)*dx
    a += InnerProduct(grad(u),beta* v)*dx
    a += mu*u*v*dx

    a.Assemble();f.Assemble()

    gfu = GridFunction(Vh)
    gfu.vec.data = a.mat.Inverse(Vh.FreeDofs()) * f.vec

    return gfu

In [107]:
gfu = H1_ellipticSolver(mesh, order=1)
Draw(gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [102]:
def Convergence_Test(h0 = 0.25, nmeshes=6, order=1, metodo='H1'):
    # solucion exacta
    uexact = sin(pi*x)*sin(pi*y)

    err_u = np.zeros(nmeshes)
    h = np.zeros(nmeshes)
    h_max = h0

    # Creamos la malla inicial
    mesh = Mesh(unit_square.GenerateMesh(maxh=h0))
    h[0] = computehmax(mesh)
    Ndof =[]
    for j in range(nmeshes):
        if metodo is 'CR':
            print('Crouzeix-Raviart method!')
            gfu = CR_ellipticSolver(mesh)
        else: 
            gfu = H1_ellipticSolver(mesh, order)
        err_u[j]  = sqrt( Integrate( InnerProduct( gfu - uexact, gfu -uexact), mesh))
        
        Ndof.append(gfu.space.ndof)
        if j < nmeshes-1:
            mesh.Refine()
            h[j+1] = computehmax(mesh)
    rate = computerate(h, err_u)
    return h, err_u, rate, gfu, Ndof

def computehmax(mesh):
    elvol = Integrate(CoefficientFunction(1),mesh,element_wise=True)
    hh = [(2*vol)**(1/2) for vol in elvol] # 3D
    return max(hh)

def computerate(h, error):
    rate = np.zeros(h.size)
    for j in range(h.size-1):
        rate[j+1] = np.log(error[j+1]/error[j])/np.log(h[j+1]/h[j])
    return rate

In [103]:
h1, err_u1, rate1, gfu1, Ndof1 = Convergence_Test(h0 = 0.25, nmeshes=6, order=1)
h2, err_u2, rate2, gfu2, Ndof2 = Convergence_Test(h0 = 0.25, nmeshes=6, order=2)
h3, err_u3, rate3, gfu3, Ndof3 = Convergence_Test(h0 = 0.25, nmeshes=6, order=3)

In [104]:
import pandas as pd
df = pd.DataFrame({'h':h1, 'Ndof k=1':Ndof1, 'error k=1':err_u1, 'order k=1':rate1,
                  'Ndof k=2':Ndof2,'error k=2':err_u2, 'order k=2':rate2,
                  'Ndof k=3':Ndof3,'error k=3':err_u3, 'order k=3':rate3})
df

,h,Ndof k=1,error k=1,order k=1,Ndof k=2,error k=2,order k=2,Ndof k=3,error k=3,order k=3
0,0.306736,26,0.058905,0.000000,85,3.706092e-03,0.000000,178,2.961737e-04,0.000000
1,0.153368,85,0.014738,1.998813,305,9.139712e-04,2.019678,661,2.567865e-05,3.527802
2,0.076684,305,0.003633,2.020183,1153,1.180343e-04,2.952943,2545,1.649792e-06,3.960213
3,0.038342,1153,0.000901,2.012055,4481,1.501218e-05,2.975000,9985,1.026949e-07,4.005848
4,0.019171,4481,0.000224,2.005455,17665,1.892500e-06,2.987769,39553,6.378081e-09,4.009098
5,0.009586,17665,0.000056,2.002540,70145,2.375376e-07,2.994065,157441,3.969659e-10,4.006035


In [108]:
def CR_ellipticSolver(mesh):
    sigma = 1.0

    Vh = FESpace("nonconforming", mesh, dirichlet='left|bottom|right|top')
    u = Vh.TrialFunction()
    v = Vh.TestFunction()

    f = LinearForm(Vh)

    f += (sigma*2*pi**2*sin(pi*x)*sin(pi*y) ) *v*dx


    a = BilinearForm(Vh)
    a += grad(u)*grad(v)*dx

    a.Assemble();f.Assemble()

    gfu = GridFunction(Vh)
    gfu.vec.data = a.mat.Inverse(Vh.FreeDofs()) * f.vec

    return gfu


In [109]:
gfu = CR_ellipticSolver(mesh)
Draw(gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [73]:
h1, err_uCR1, rateCR1, gfuCR1, NdofCR1 = Convergence_Test(h0 = 0.25, nmeshes=6, order=1, metodo='CR')

Crouzeix-Raviart method!
Crouzeix-Raviart method!
Crouzeix-Raviart method!
Crouzeix-Raviart method!
Crouzeix-Raviart method!
Crouzeix-Raviart method!


In [75]:
dfCR = pd.DataFrame({'h':h1, "Ndof":NdofCR1, 'error k=1':err_uCR1, 'order k=1':rateCR1})
dfCR

,h,Ndof,error k=1,order k=1
0,0.306736,59,0.039923,0.000000
1,0.153368,279,0.011170,1.837575
2,0.076684,1127,0.002869,1.960808
3,0.038342,4455,0.000723,1.988908
4,0.019171,17639,0.000181,1.997224
5,0.009586,70119,0.000045,1.999474


In [92]:
mesh = Mesh(unit_cube.GenerateMesh(maxh=0.1))
Vh = FESpace("nonconforming", mesh, dirichlet='front|left|back|right')
u = Vh.TrialFunction()
v = Vh.TestFunction()

sigma=1
f = LinearForm(Vh)
f += (sigma*2*pi**2*sin(pi*x)*sin(pi*y) ) *v*dx
a = BilinearForm(Vh)
a += grad(u)*grad(v)*dx

a.Assemble();f.Assemble()

gfu = GridFunction(Vh)
gfu.vec.data = a.mat.Inverse(Vh.FreeDofs()) * f.vec

In [93]:
Draw(gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene